In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
def assign(dataset, medoids):
    """
    Time complexity: O(K*N)
    """
    K = len(medoids)
    clusters = [[] for _ in range(K)]
    assigned = [np.sum((np.tile(d, (K, 1)) - medoids)**2,
                       axis=1).argmin() for d in dataset]
    for i, c in enumerate(assigned):
        clusters[c].append(dataset[i])
    return assigned, list(map(np.array, clusters))


def cost(clusters, seeds):
    """
    Time complexity: O(N)
    """
    K = len(seeds)
    c = 0
    for k in range(K):
        cluster = clusters[k]
        c += np.sum((np.tile(seeds[k], (len(cluster), 1)) - cluster)**2)
    return c

In [ ]:
K = 3
columns = ['instrumentalness', 'liveness']
df = pd.read_csv('./dataset/spotify.csv')[columns]
dataset = df.to_numpy()
seeds = [x for x in range(0, len(df), len(df)//K)][:K]
medoids = dataset[seeds]

In [ ]:
# PAM (Partitioning Around Medoids) Method

seeds = medoids
k_index, clusters = assign(dataset, seeds)
t = 0
old_cost = cost(clusters, seeds)
changed = True

while changed:
    changed = False
    t += 1
    for k, o in tqdm(zip(k_index, dataset), desc=str(t), total=len(dataset)):
        seed = seeds[k]
        new_seeds = seeds.copy()
        new_seeds[k] = o
        _k_index, new_clusters = assign(dataset, new_seeds)
        new_cost = cost(new_clusters, new_seeds)
        if old_cost > new_cost:
            seeds = new_seeds
            clusters = new_clusters
            new_k_index = _k_index
            old_cost = new_cost
            changed = True
    k_index = new_k_index

medoids = seeds

In [ ]:
for c in clusters:
    plt.scatter(c.T[0], c.T[1], s=1)
for m in medoids:
    plt.scatter(m[0], m[1], s=20, c='black')
plt.xlabel(columns[0])
plt.ylabel(columns[1])
plt.show()